In [1]:
import re
import random
import time
from statistics import mode
import json 

from PIL import Image
import numpy as np
import pandas
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms

In [2]:
import matplotlib.pyplot as plt
from einops.layers.torch import Rearrange
from einops import rearrange
from einops import repeat

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"device: {device}")

device: cuda


In [4]:
def process_text_adv(text):
    # lowercase
    text = text.lower()

    # 数詞を数字に変換
    num_word_to_digit = {
        'zero': '0', 'one': '1', 'two': '2', 'three': '3', 'four': '4',
        'five': '5', 'six': '6', 'seven': '7', 'eight': '8', 'nine': '9',
        'ten': '10', 
    }
    for word, digit in num_word_to_digit.items():
        text = text.replace(word, digit)

    # 小数点のピリオドを削除
    text = re.sub(r'(?<!\d)\.(?!\d)', '', text)

    # 冠詞の削除
    text = re.sub(r'\b(a|an|the)\b', '', text)

    # 短縮形のカンマの追加
    contractions = {
        "dont": "don't", "isnt": "isn't", "arent": "aren't", "wont": "won't",
        "cant": "can't", "wouldnt": "wouldn't", "couldnt": "couldn't", "whats": "what's",
        "thats": "that's", "whos": "who's", "wheres": "where's", "whens": "when's",
        "please":"", "could you": "can you", "could i": "can i", "could we": "can we",
    }

    contractions_2 = {
        "theatre" : "theater", "colour" : "color", "centre" : "center", "favourite" : "favorite",
        "travelling" : "traveling", "counselling" : "counseling", "metre" : "meter",
        "cancelled" : "canceled", "labour" : "labor", "organisation" : "organization",
        "calibre" : "caliber", "cheque" : "check", "manoeuvre" : "maneuver",
        "neighbour" : "neighbor", "grey" : "gray", "dialogue" : "dialog",
    }

    # contractions_3 = {
    #     "what is": "what's", "who is": "who's", "where is": "where's", "when is": "when's",
    #     "how is": "how's", "it is": "it's", "he is": "he's", "she is": "she's",
    #     "that is": "that's", "there is": "there's", "here is": "here's",
    #     "i am": "i'm", "you are": "you're", "we are": "we're", "they are": "they're",
    #     "i have": "i've", "you have": "you've", "we have": "we've", "they have": "they've",
    #     "i will": "i'll", "you will": "you'll",
    # }
    contractions_3 = {
        "what's" : "what is", "who's" : "who is", "where's" : "where is", "when's" : "when is",
        "how's" : "how is", "it's" : "it is", "he's" : "he is", "she's" : "she is",
        "that's" : "that is", "there's" : "there is", "here's" : "here is",
        "i'm" : "i am", "you're" : "you are", "we're" : "we are", "they're" : "they are",
        "i've" : "i have", "you've" : "you have", "we've" : "we have", "they've" : "they have",
        "i'll" : "i will", "you'll" : "you will",
    }
 
    for contraction, correct in contractions.items():
        text = text.replace(contraction, correct)
    for contraction, correct in contractions_2.items():
        text = text.replace(contraction, correct)
    for contraction, correct in contractions_3.items():
        text = text.replace(contraction, correct)

    # 句読点をスペースに変換
    text = re.sub(r"[^\w\s':]", ' ', text)

    # 句読点をスペースに変換
    text = re.sub(r'\s+,', ',', text)

    # 連続するスペースを1つに変換
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [5]:
def process_text_adv(text):
    # lowercase
    text = text.lower()

    # 数詞を数字に変換
    num_word_to_digit = {
        'zero': '0', 'one': '1', 'two': '2', 'three': '3', 'four': '4',
        'five': '5', 'six': '6', 'seven': '7', 'eight': '8', 'nine': '9',
        'ten': '10', 
    }
    for word, digit in num_word_to_digit.items():
        text = text.replace(word, digit)

    # 小数点のピリオドを削除
    text = re.sub(r'(?<!\d)\.(?!\d)', '', text)

    # 冠詞の削除
    text = re.sub(r'\b(a|an|the)\b', '', text)

    # 短縮形のカンマの追加
    contractions = {
        "dont": "don't", "isnt": "isn't", "arent": "aren't", "wont": "won't",
        "cant": "can't", "wouldnt": "wouldn't", "couldnt": "couldn't", "whats": "what's",
        "thats": "that's", "whos": "who's", "wheres": "where's", "whens": "when's",
        "please":"", "could you": "can you", "could i": "can i", "could we": "can we",
    }

    contractions_2 = {
        "theatre" : "theater", "colour" : "color", "centre" : "center", "favourite" : "favorite",
        "travelling" : "traveling", "counselling" : "counseling", "metre" : "meter",
        "cancelled" : "canceled", "labour" : "labor", "organisation" : "organization",
        "calibre" : "caliber", "cheque" : "check", "manoeuvre" : "maneuver",
        "neighbour" : "neighbor", "grey" : "gray", "dialogue" : "dialog",
    }

    # contractions_3 = {
    #     "what is": "what's", "who is": "who's", "where is": "where's", "when is": "when's",
    #     "how is": "how's", "it is": "it's", "he is": "he's", "she is": "she's",
    #     "that is": "that's", "there is": "there's", "here is": "here's",
    #     "i am": "i'm", "you are": "you're", "we are": "we're", "they are": "they're",
    #     "i have": "i've", "you have": "you've", "we have": "we've", "they have": "they've",
    #     "i will": "i'll", "you will": "you'll",
    # }
    contractions_3 = {
        "what's" : "what is", "who's" : "who is", "where's" : "where is", "when's" : "when is",
        "how's" : "how is", "it's" : "it is", "he's" : "he is", "she's" : "she is",
        "that's" : "that is", "there's" : "there is", "here's" : "here is",
        "i'm" : "i am", "you're" : "you are", "we're" : "we are", "they're" : "they are",
        "i've" : "i have", "you've" : "you have", "we've" : "we have", "they've" : "they have",
        "i'll" : "i will", "you'll" : "you will",
    }
 
    for contraction, correct in contractions.items():
        text = text.replace(contraction, correct)
    for contraction, correct in contractions_2.items():
        text = text.replace(contraction, correct)
    for contraction, correct in contractions_3.items():
        text = text.replace(contraction, correct)

    # 句読点をスペースに変換
    text = re.sub(r"[^\w\s':]", ' ', text)

    # 句読点をスペースに変換
    text = re.sub(r'\s+,', ',', text)

    # 連続するスペースを1つに変換
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [6]:
def process_text(text):
    # lowercase
    text = text.lower()

    # 数詞を数字に変換
    num_word_to_digit = {
        'zero': '0', 'one': '1', 'two': '2', 'three': '3', 'four': '4',
        'five': '5', 'six': '6', 'seven': '7', 'eight': '8', 'nine': '9',
        'ten': '10'
    }
    for word, digit in num_word_to_digit.items():
        text = text.replace(word, digit)

    # 小数点のピリオドを削除
    text = re.sub(r'(?<!\d)\.(?!\d)', '', text)

    # 冠詞の削除
    text = re.sub(r'\b(a|an|the)\b', '', text)

    # 短縮形のカンマの追加
    contractions = {
        "dont": "don't", "isnt": "isn't", "arent": "aren't", "wont": "won't",
        "cant": "can't", "wouldnt": "wouldn't", "couldnt": "couldn't"
    }
    for contraction, correct in contractions.items():
        text = text.replace(contraction, correct)

    # 句読点をスペースに変換
    text = re.sub(r"[^\w\s':]", ' ', text)

    # 句読点をスペースに変換
    text = re.sub(r'\s+,', ',', text)

    # 連続するスペースを1つに変換
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [7]:
class VQADataset(torch.utils.data.Dataset):
    def __init__(self, df_path, image_dir, transform=None, answer=True):
        self.transform = transform  # 画像の前処理
        self.image_dir = image_dir  # 画像ファイルのディレクトリ
        self.df = pandas.read_json(df_path)  # 画像ファイルのパス，question, answerを持つDataFrame
        self.answer = answer

        # question / answerの辞書を作成
        self.question2idx = {}
        self.answer2idx = {}
        self.idx2question = {}
        self.idx2answer = {}
        self.questionInitialWords = dict()
        c1 = c2 = c3 = c4 = c5 = c6 = c7 = 0

        # 質問文に含まれる単語を辞書に追加
        for question in self.df["question"]:
            question = process_text_adv(question)
            words = question.split(" ")
            
            for idex, word in enumerate(words):
                if idex == 0:
                    if word not in self.questionInitialWords:
                        self.questionInitialWords[word] = 1
                    else:
                        self.questionInitialWords[word] += 1            
                if word not in self.question2idx:
                    self.question2idx[word] = len(self.question2idx)
        self.idx2question = {v: k for k, v in self.question2idx.items()}  # 逆変換用の辞書(question)
        print(self.questionInitialWords)
        print(len(self.questionInitialWords))

        if self.answer:
            # 回答に含まれる単語を辞書に追加
            for answers in self.df["answers"]:
                for answer in answers:
                    word = answer["answer"]
                    word = process_text(word)
                    if word not in self.answer2idx:
                        self.answer2idx[word] = len(self.answer2idx)
            self.idx2answer = {v: k for k, v in self.answer2idx.items()}  # 逆変換用の辞書(answer)

            # print(f"answer{self.idx2answer}")

    def update_dict(self, dataset):
        """
        検証用データ，テストデータの辞書を訓練データの辞書に更新する．

        Parameters
        ----------
        dataset : Dataset
            訓練データのDataset
        """
        self.question2idx = dataset.question2idx
        self.answer2idx = dataset.answer2idx
        self.idx2question = dataset.idx2question
        self.idx2answer = dataset.idx2answer

    def __getitem__(self, idx, isKL=False):
        """
        対応するidxのデータ（画像，質問，回答）を取得．

        Parameters
        ----------
        idx : int
            取得するデータのインデックス

        Returns
        -------
        image : torch.Tensor  (C, H, W)
            画像データ
        question : torch.Tensor  (vocab_size)
            質問文をone-hot表現に変換したもの
        answers : torch.Tensor  (n_answer)
            10人の回答者の回答のid
        mode_answer_idx : torch.Tensor  (1)
            10人の回答者の回答の中で最頻値の回答のid
        """
        image = Image.open(f"{self.image_dir}/{self.df['image'][idx]}")
        image = self.transform(image)
        question = np.zeros(len(self.idx2question) + 1)  # 未知語用の要素を追加
        question_words = self.df["question"][idx].split(" ")

        for word in question_words:
            try:
                question[self.question2idx[word]] = 1  # one-hot表現に変換
            except KeyError:
                question[-1] = 1  # 未知語

        if self.answer:
            answers_prevector = np.zeros(len(self.answer2idx))
            answers = [self.answer2idx[process_text(answer["answer"])] for answer in self.df["answers"][idx]]
            for answer in answers:
                answers_prevector[answer] += 1
            sum_answers = max(np.sum(answers_prevector), 1e-10)
            answers_vector = answers_prevector / sum_answers
            mode_answer_idx = mode(answers)  # 最頻値を取得（正解ラベル）

            return image, torch.Tensor(question), torch.Tensor(answers), int(mode_answer_idx), torch.Tensor(answers_vector)

        else:
            return image, torch.Tensor(question)

    def __len__(self):
        return len(self.df)

In [8]:
transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [9]:
# "answers":{"0":[{"answer_confidence":"yes","answer":"beef chuck steak"},{"answer_confidence":"yes","answer":"beef chuck steak"},{"answer_confidence":"yes","answer":"beef chuck steak"},{"answer_confidence":"yes","answer":"beef chuck steak"},{"answer_confidence":"yes","answer":"flat iron beef chuck steak"},{"answer_confidence":"yes","answer":"beef chuck steak"},{"answer_confidence":"yes","answer":"steak"},{"answer_confidence":"yes","answer":"flat iron beef chuck steak"},{"answer_confidence":"yes","answer":"beef chuck steak"},{"answer_confidence":"yes","answer":"beef chuck steak"}],"1":[{"answer_confidence":"yes","answer":"unanswerable"},{"answer_confidence":"yes","answer":"unanswerable"},{"answer_confidence":"yes","answer":"unanswerable"},{"answer_confidence":"yes","answer":"candle"},{"answer_confidence":"no","answer":"unanswerable"},{"answer_confidence":"maybe","answer":"unanswerable"},{"answer_confidence":"yes","answer":"unanswerable"},{"answer_confidence":"yes","answer":"unanswerable"},{"answer_confidence":"no","answer":"unanswerable"},{"answer_confidence":"yes","answer":"unanswerable"}],

In [10]:
import json

# COCO_val2014_000000393075

answer_type = dict()
extra_answer_output = dict()

print("test")
# データセットの読み込み
with open('./extra_data/train_question.json', 'r') as f:
    question_data = json.load(f)

with open('./extra_data/train_answer.json', 'r') as f:
    answer_data = json.load(f)

extra_questions_data = question_data["questions"]
extra_answer_data = answer_data["annotations"]

test


In [11]:
print("test")
print(len(extra_questions_data))
print(extra_questions_data[0])

start_number = 19873

question_map_dict = dict()
image_dict = dict()
question_dict = dict()
final_dict = dict()

for data in extra_questions_data:
    original_image_id = data["image_id"]
    question_id = data["question_id"]
    image_name = "COCO_train2014_" + str(original_image_id).zfill(12) + ".jpg"
    
    question_map_dict[question_id] = start_number
    image_dict[start_number] = image_name
    re_question_id = start_number
    
    question = data["question"]
    question_dict[re_question_id] = question

    start_number += 1


final_dict["image"] = image_dict
final_dict["question"] = question_dict
print(extra_answer_data[0])

print("test")

test
214354
{'image_id': 262148, 'question': 'Where is he looking?', 'question_id': 262148000}
{'question_type': 'none of the above', 'multiple_choice_answer': 'down', 'answers': [{'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 1}, {'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 2}, {'answer': 'at table', 'answer_confidence': 'yes', 'answer_id': 3}, {'answer': 'skateboard', 'answer_confidence': 'yes', 'answer_id': 4}, {'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 5}, {'answer': 'table', 'answer_confidence': 'yes', 'answer_id': 6}, {'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 7}, {'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 8}, {'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 9}, {'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 10}], 'image_id': 262148, 'answer_type': 'other', 'question_id': 262148000}
test


In [12]:
# with open('./extra_data/test_question.json', 'w') as f:
#     json.dump(question_dict, f)

In [13]:
# with open('./extra_data/test_image.json', 'w') as f:
#     json.dump(image_dict, f)

In [14]:
min_image_id = 999999999999999999

for data in extra_answer_data:
    question_id = data["question_id"]

    re_question_id = question_map_dict[question_id]

    image_id = data["image_id"]
    if image_id < min_image_id:
        min_image_id = image_id
    answer_list = data['answers']
    replace_answer_list = []
    for answer in answer_list:
        answer_data = dict()
        answer_data["answer_confidence"] = answer["answer_confidence"]
        answer_data["answer"] = answer["answer"]
        answer = answer_data
        replace_answer_list.append(answer_data)
    extra_answer_output[re_question_id] = replace_answer_list

final_dict["answers"] = extra_answer_output

print(f"{extra_answer_output[19873]}")
print(f"min_image_id: {min_image_id}")

[{'answer_confidence': 'yes', 'answer': 'down'}, {'answer_confidence': 'yes', 'answer': 'down'}, {'answer_confidence': 'yes', 'answer': 'at table'}, {'answer_confidence': 'yes', 'answer': 'skateboard'}, {'answer_confidence': 'yes', 'answer': 'down'}, {'answer_confidence': 'yes', 'answer': 'table'}, {'answer_confidence': 'yes', 'answer': 'down'}, {'answer_confidence': 'yes', 'answer': 'down'}, {'answer_confidence': 'yes', 'answer': 'down'}, {'answer_confidence': 'yes', 'answer': 'down'}]
min_image_id: 42


In [15]:
# with open('./extra_data/test.json', 'w') as f:
#     json.dump(extra_answer_output, f, indent=2)

# with open('./extra_data/final_valie.json', 'w') as f:
#     json.dump(final_dict, f)

In [16]:
import pandas

In [41]:
def original_data(df_path, image_dir="", transform=None, answer=False):

    df = pandas.read_json(df_path)

    # question / answerの辞書を作成
    question2idx = {}
    answer2idx = {}
    questionInitialWords = dict()
    c1 = c2 = c3 = c4 = c5 = c6 = c7 = 0

    first_three_words = []
    first_four_words = []

    # 質問文に含まれる単語を辞書に追加
    for question in df["question"]:
        question = process_text_adv(question)
        words = question.split(" ")

        if len(words) > 2:
            first_three_words.append(words[0] + " " + words[1] + " " + words[2])
            if len(words) > 3:
                first_four_words.append(words[0] + " " + words[1] + " " + words[2] + " " + words[3])
            
        for idex, word in enumerate(words):
            if idex == 0:
                if word not in questionInitialWords:
                    questionInitialWords[word] = 1
                else:
                    questionInitialWords[word] += 1            
            if word not in question2idx:
                question2idx[word] = len(question2idx)
    idx2question = {v: k for k, v in question2idx.items()}  # 逆変換用の辞書(question)

    if answer:
        # 回答に含まれる単語を辞書に追加
        for answers in df["answers"]:
            for answer in answers:
                word = answer["answer"]
                word = process_text(word)
                if word not in answer2idx:
                    answer2idx[word] = len(answer2idx)
        idx2answer = {v: k for k, v in answer2idx.items()}  # 逆変換用の辞書(answer)

    return first_three_words, first_four_words


In [34]:
from tqdm import tqdm

In [46]:
three_words_1, four_words_1 = original_data('./data/train.json')
three_words_2, four_words_2 = original_data('./data/valid.json')

counter = 0
counter_four = 0
counter_only_three = 0

dest = len(question_dict)

start_number = 19873

selected_question = []

for idx in tqdm(range(dest)):
    index = start_number + idx

    new_first_three_words = ""
    question_state = question_dict[index]
    question_state = process_text_adv(question_state)
    
    question_list = question_state.split(" ")
    if len(question_list) <= 2:
        continue 
    
    new_first_three_words = question_list[0] + " " + question_list[1] + " " + question_list[2]
    new_first_four_words = ""
    if len(question_list) > 3:
        new_first_four_words = question_list[0] + " " + question_list[1] + " " + question_list[2] + " " + question_list[3]


    if new_first_three_words in three_words_2:
        counter += 1
        if len(question_list) > 3:
            if new_first_four_words in four_words_1:
                counter_four += 1
        else:
            counter_only_three += 1
        selected_question.append(index)
        continue

    if new_first_three_words in three_words_1:    
        counter += 1
        if len(question_list) > 3:
            if new_first_four_words in four_words_2:
                counter_four += 1
        else:
            counter_only_three += 1
        selected_question.append(index)
        continue

print(f"counter: {counter}")
print(f"counter four: {counter_four}")
print(f"counter only three: {counter_only_three}")

100%|██████████| 214354/214354 [00:37<00:00, 5677.25it/s]

counter: 75567
counter four: 18647
counter only three: 1770


In [ ]:
train_dataset = VQADataset(df_path="./data/train.json", image_dir="./data/train", transform=transform_test)

{'what': 13937, 'maybe': 2, 'can': 1410, 'is': 949, 'tell': 186, 'when': 64, 'my': 9, 'you': 18, 'how': 366, 'for': 48, 'do': 98, 'i': 320, 'alright': 22, 'kind': 9, 'hi': 192, 'guy': 4, 'in': 26, 'would': 17, 'this': 192, 'yes': 75, 'which': 181, 'dinner': 2, 'describe': 86, 'who': 84, 'fall': 1, 'will': 17, 'now': 12, 'okay': 75, 'shut': 1, 'dishwasher': 1, 'read': 54, 'if': 18, 'uncle': 1, 'specifically': 5, 'grab': 2, 'oh': 11, 'are': 106, 'hiis': 1, 'hello': 98, 'were': 1, "let's": 7, "i'd": 8, 'say': 4, 'test': 6, 'does': 154, 'where': 124, 'so': 16, 'just': 18, 'end': 1, 'and': 12, 'calorie': 1, 'first': 2, 'slaver': 1, 'good': 14, 'brand': 3, 'little': 1, 'ask': 1, 'as': 1, 'wonder': 1, 'it': 28, 'security': 1, 'flavor': 5, 'ok': 47, 'there': 21, 'trying': 8, 'thank': 17, 'we': 6, 'brahma': 1, 'sparks': 2, 'identify': 24, 'sending': 1, 'hey': 31, 'ty': 1, 'next': 1, 'yeah': 17, 'last': 4, 'they': 2, 'bus': 1, 'screen': 3, 'video': 1, 'well': 5, 'all': 5, 'starting': 2, 'may': 5